# 0. selenium 설치

In [1]:
!pip install chromedriver_autoinstaller

In [2]:
!pip install selenium

# 1. 라이브러리 불러오기

In [3]:
# 라이브러리
import numpy as np
import pandas as pd
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함


from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

import chromedriver_autoinstaller

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm


# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time


# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 크롤링 (시즌 & 클럽 경력 추가 part)

##### 참고


- find_elements_by_tag_name:          Copy Element의 tag
- find_elements_by_xpath:             Copy XPath 또는 Copy full XPath
- find_elements_by_id:                Copy Element의 id attribute
- find_elements_by_class_name:        Copy Element의 class attribute
- find_elements_by_link_text:         Copy Element의 text
- find_elements_by_partial_link_text: Copy Element의 text중 일부
- find_elements_by_css_selector:      Copy selector


- '요소' 클릭:        	.click('')   
- '요소' 더블 클릭:	     .double_click('')   
- '텍스트' 입력:	         .send_keys('')  
- '요소'에 '텍스트' 입력:	  .send_keys_to_element('','')  
- '요소'로 마우스 이동:	  .move_to_element('')
- 스크롤 내리기:	         .execute_script("window.scrollto();")
- 실행:                  .perform()

> 각 선수별 시즌, 클럽 경력을 가져온 뒤, 'name'과 세부 능력치 기준으로 merge 예정!

##### 크롤링 리스트 만들기

In [41]:
# 선수명 csv파일 불러오기
player_name = pd.read_csv('./data/player_info/player.csv')
# 리스트화
search_name_list = list(player_name['play_name'])
# 중복값 제거
search_name_list = list(set(search_name_list))

##### 크롤링 기록

> 정지된 기록

<br>

|구간|원인|비고|
|---|---|---|
|0 ~ 1233|        * 카톡 로그아웃||
|1234 ~ 14067|    * 검색 결과 오류  |    +동명이인 1차 업데이트: '동명이인' 출력|
|14068 ~ 20243|   * 검색 결과 오류 |     +동명이인 2차 업데이트: +'temp_name'|
|20244 ~ 22505|   * 검색 결과 오류|      +동명이인 3차 업데이트: +'search_name'|
|22506 ~ 23449|   * 검색 결과 오류|  'N. 메사투' 검색결과 없음 (number = 23450)|
|23451 ~ 25034|   * 검색 결과 오류||
|25035 ~ 30961|   * 사용할 수 없는 페이지|+'검색 결과 오류' 예외처리 업데이트|
|30962 ~ 32411|   * 검색어 입력 에러||
|32412 ~ 36644|   * 완료!||

<br><br>

> 후처리 과정에서의 편의를 위해 다시 크롤링 진행 (~ 22505)

by 동명이인 3차 업데이트 ver.

<br>

|구간|원인|비고|
|---|---|---|
| 0 ~ 7565 |* 검색어 입력 오류|+ 오류 발생시 초기화 버튼 누르기|
| 7566 ~ 16057 |* 사용할 수 없는 페이지| + 페이지 오류시 대처 코드 추가|
| 16058 ~ 16394 |* 클릭 오류||
| 16395 ~ 17984 | * 검색결과 너무 많음 (검색명: 안드레)||
| 17985 ~ 19444 |* 사용할 수 없는 페이지||
| 19445 ~ 22505|완료!| + 동명이인 2차 업데이트 구간까지 크롤링|


##### 정지된 위치 확인 코드

In [42]:
# 정지된 위치 확인
def find_end(last_search_name):
    for i in range(len(search_name_list)):
        if search_name_list[i] == last_search_name:
            print(i)


find_end('리다')

32153


##### * 사용할 함수

In [43]:
# 크롬 윈도우 설정
def open_chrome():
    # 크롬 드라이버 위치 설정
    chrome_path = chromedriver_autoinstaller.install()


    # 크롬 옵션
    options = webdriver.ChromeOptions()
    # 크롬 윈도우 사이즈 조절
    options.add_argument("--window-size=800,1400")

    driver = webdriver.Chrome(chrome_path, options=options)

    # 실행할 시간 랜덤값 지정
    A = np.random.randint(5,7)
    
    return driver , A



# 클럽 활동 연도 & 클럽 팀 합치기
def year_and_team_merge(temp_club_career):
    global temp_club_career_2
    temp_club_career_2 = []
    year = []
    team = []
    

    for i in range(1,len(temp_club_career.split('\n'))):
        if i % 2 == 0:
            year.append(temp_club_career.split('\n')[i])
        else:
            team.append(temp_club_career.split('\n')[i])


    for i in range(len(year)):
        temp_club_career_2.append(year[i]+' / '+team[i])
        
    return temp_club_career_2



# 크롤링
def crawling(start_num,end_num):
   # chrome 창 열기
   driver, A = open_chrome()

   ## 선수 정보 >> 각 선수의 시즌 & 클럽 경력 크롤링
   # 선수 정보 리스트
   name = []
   season = []
   club_career = []
   pay_side = []
   overall = []
   Sprint_speed = []
   Acceleration =[]
   Strength = []
   Stamina = []
   error_name = []

   # 홈페이지 열기
   url = 'https://fifaonline4.nexon.com/datacenter'
   driver.get(url)

   # # 팝업창 닫기
   # driver.find_element_by_xpath('//*[@id="wrapper"]/div[1]/a/span').click()
   # time.sleep(A)

   # 정지된 위치부터 다시 시작 
   for search_name in tqdm(search_name_list[start_num:end_num+1]):
      try:
         # 선수명 입력 & 검색
         driver.find_element_by_class_name('ui-autocomplete-input').click()
         driver.find_element_by_class_name('ui-autocomplete-input').send_keys(search_name)
         driver.find_element_by_class_name('btn_search').click()
         time.sleep(2.5)


         # 세부 설명 들어가기
         driver.find_element_by_class_name('name').click()
         driver.find_element_by_class_name('btn_detail_link').click()
         time.sleep(0.5)
         
         
         # 창 변경
         driver.switch_to.window(driver.window_handles[-1])
         time.sleep(1)
         
         
         # 클럽 경력 추출
         temp_club_career = driver.find_element_by_xpath('//*[@id="middle"]/div/div/div[4]/div[1]/div[2]/div[2]').text
         temp_club_career_2 = year_and_team_merge(temp_club_career)
         
         
         # 세부 설명 나오기
         driver.close()
         time.sleep(0.5)

         
         # 창 변경
         driver.switch_to.window(driver.window_handles[0])
         time.sleep(0.5)



         # 검색된 선수 숫자 세기
         player_list = driver.find_elements_by_xpath('//*[@id="divPlayerList"]')
         player_list_num = player_list[0].text.count(f'{search_name}')



         # 시즌, 이름, 급여, 오버롤, 속력, 가속력, 몸싸움, 스태미너 추출
         for num in range(1,player_list_num+1):
            # 이름 저장
            temp_name = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[1]/div/div[4]/div[2]').text
            name.append(temp_name)
            
            # 클럽 경력 저장
            # 같이 검색된 동명이인 처리
            if search_name == temp_name:
               club_career.append(temp_club_career_2)
            else:
               club_career.append('동명이인'+search_name)

            # 시즌 저장
            temp_season = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[1]/div/div[4]/div[1]/img')
            temp_season = temp_season.get_attribute('src').split('/')[-1].replace('.png','')
            season.append(temp_season)
      
            # 급여 저장
            temp_pay_side = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[2]/span').text
            pay_side.append(temp_pay_side)
         
            # 오버롤 저장
            temp_overall = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[3]/span/span').text
            overall.append(temp_overall)
            
            # 속력 저장
            temp_Sprint_speed = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[4]/span/span').text
            Sprint_speed.append(temp_Sprint_speed)
            
            # 가속력 저장
            temp_Acceleration = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[5]/span/span').text
            Acceleration.append(temp_Acceleration)
            
            # 몸싸움 저장
            temp_Strength = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[6]/span/span').text
            Strength.append(temp_Strength)
            
            # 스태미너 저장
            temp_Stamina = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[7]/span/span').text
            Stamina.append(temp_Stamina)
         

         # 검색어 초기화 
         driver.find_element_by_class_name('btn_reset').click()
         time.sleep(1)
      
      except:
         if driver.find_element_by_xpath('//*[@id="divPlayerList"]/div/span').text == '검색 결과가 없습니다.':
            error_name.append(search_name)
            print(f'error(검색 결과가 없습니다.): {search_name}')
            
            # 검색어 초기화 
            driver.find_element_by_class_name('btn_reset').click()
            pass
         
         elif driver.find_element_by_xpath('//*[@id="ErrorMessage"]').text == '사용할 수 없는 페이지입니다.':
            error_name.append(search_name)
            print(f'error(사용할 수 없는 페이지입니다.): {search_name}')
            
            # 창 닫기
            driver.close()
            time.sleep(0.5)

            # 창 변경
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(0.5)
            
            # 검색어 초기화 
            driver.find_element_by_class_name('btn_reset').click()
            pass
            
         else:
            error_name.append(search_name)
            print(f'error: {search_name}')
            break

   return name,season,club_career,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,error_name



# dict로 변형 & 데이터 프레임화
def dict_to_df(name,season,club_career,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina):
    dict_temp = {  'name': [val for val in name],
                'pay_side': [val for val in pay_side],
                'overall': [val for val in overall],
                'Sprint_speed': [val for val in Sprint_speed],
                'Acceleration': [val for val in Acceleration],
                'Strength': [val for val in Strength],
                'Stamina': [val for val in Stamina],
                'season': [val for val in season],
                'club_career': [val for val in club_career]                            
    }
    columns = ['name','pay_side','overall','Sprint_speed','Acceleration','Strength','Stamina','season','club_career']

    player_info_plus = pd.DataFrame(data = dict_temp, columns=columns)
    
    return player_info_plus
 
 
 
def merge_csv(player_info_plus):
    # 이전 csv 파일 불러오기
    player_info_plus_before = pd.read_csv('./data/player_info/player_info_plus.csv')


    # 데이터 프레임 병합
    # 이전 csv 파일 불러오기
    New_player_info_plus = pd.concat([player_info_plus_before,player_info_plus], ignore_index=True)
    New_player_info_plus

    # csv 파일로 저장
    New_player_info_plus.to_csv("./data/player_info/player_info_plus.csv", encoding='utf-8-sig', index = False)
    
    return New_player_info_plus 

##### 실행

In [44]:
start_num = find_end('리다')+1
end_num = 36644

# 크롤링
name,season,club_career,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,error_name = crawling(start_num,end_num)
player_info_plus = dict_to_df(name,season,club_career,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina)
print(error_name)

# 데이터 확인
player_info_plus

  0%|          | 0/5 [00:00<?, ?it/s]

[]


,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,벤 폴라미,5,59,74,72,69,65,LIVE,동명이인벤 폴
1,벤 폴,5,57\n(56),26,25,63,22,LIVE,[]
2,리 캠프,5,60,38,34,65,36,19,[]
3,리 캠프,5,57,35,30,65,36,21,[]
4,로니 슈왈츠,5,60,64,62,67,63,21,[]
5,데이비드 폭스,5,59,42,56,55,59,18,[]
6,베람 카얄,5,65,59,60,59,63,19,[]


In [45]:
# 이전 데이터와 새로운 데이터 병합
data = merge_csv(player_info_plus)
data

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,슈테펜 호프만,5,62,39,51,59,38,17,[]
1,조 에드워즈,5,60,62,64,65,83,LIVE,[]
2,J. 피에라치,5,60,59,55,68,66,18,[]
3,김도훈,21,99,96,89,106,90,TKL,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
4,김도훈,17,88,84,77,97,86,TKI,"['1995 ~ 2002 / 성남 FC', '1998 ~ 1999 / 전북 현대 모..."
...,...,...,...,...,...,...,...,...,...
104288,리 캠프,5,60,38,34,65,36,19,[]
104289,리 캠프,5,57,35,30,65,36,21,[]
104290,로니 슈왈츠,5,60,64,62,67,63,21,[]
104291,데이비드 폭스,5,59,42,56,55,59,18,[]


# 3. 데이터 후처리

##### * 사용할 함수

In [136]:
# 클럽 커리어까지 완전히 일치하는 중복값 처리
def drop_duplicate(data):
    data.drop_duplicates(subset = ['name',
                                'pay_side',
                                'overall',
                                'Sprint_speed',
                                'Acceleration',
                                'Strength',
                                'Stamina',
                                'season',
                                'club_career'], keep = 'last', inplace = True) 

    return data



# 라이브 부스트 받기 전/후 차이 구하기
def find_gap(live_boost_data,num):
    LBD = live_boost_data['overall'].iloc[num].replace(')','').replace('(','').replace('\n','-')
    LBD_split = LBD.split('-')
    original_overall = LBD_split[1]
    gap = int(LBD_split[0]) - int(LBD_split[1])
    
    return gap, original_overall



# 라이브 부스트 받기 전 값으로 입력해주기
def remove_live_boost(live_boost_data,original_overall,gap,num):
    live_boost_data['pay_side'].iloc[num] = str(live_boost_data['pay_side'].iloc[num])
    live_boost_data['overall'].iloc[num] = str(original_overall)
    live_boost_data['Sprint_speed'].iloc[num] = str(int(live_boost_data['Sprint_speed'].iloc[num]) - gap)
    live_boost_data['Acceleration'].iloc[num] = str(int(live_boost_data['Acceleration'].iloc[num]) - gap)
    live_boost_data['Strength'].iloc[num] = str(int(live_boost_data['Strength'].iloc[num]) - gap)
    live_boost_data['Stamina'].iloc[num] = str(int(live_boost_data['Stamina'].iloc[num]) - gap)



# 클럽 커리어 정리
def mk_club_career(data):
    # 동명이인 선수 데이터 추출 (3-3. 에서 사용)
    data_same_name = data[data.club_career.str.contains('동명이인')]

    # 동명이인 선수를 제외한 선수 데이터 추출 
    data = data[~data.club_career.str.contains('동명이인')]
    
    
    for index in tqdm(range(len(data.club_career))):
        data.club_career.iloc[index] = data.club_career.iloc[index].replace('[]','-').replace('[','').replace(']','')
    
    return data, data_same_name

##### 실행

In [129]:
# # 데이터 불러오기
# data = pd.read_csv('./data/player_info/player_info_plus.csv')

# 4. 과정을 마치고 온 경우
data = pd.read_csv('./data/player_info/player_info_plus_final.csv')

data

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,슈테펜 호프만,5,62,39,51,59,38,17,-
1,J. 피에라치,5,60,59,55,68,66,18,-
2,누어 허신,5,55,56,61,55,66,19,-
3,마오하오위,5,49,69,60,35,49,20,-
4,마이크 고메스,5,57,65,61,67,70,19,-
...,...,...,...,...,...,...,...,...,...
50878,정우영,14,86,94,98,73,83,19NG,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."
50879,정우영,14,83,73,67,86,87,2019KFA,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."
50880,정우영,9,76,88,88,64,75,21PLA,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."
50881,정우영,5,67\n(66),77,76,52,83,LIVE,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."


In [104]:
# 결측값 확인
data.isna().sum()

name            0
pay_side        0
overall         0
Sprint_speed    0
Acceleration    0
Strength        0
Stamina         0
season          0
club_career     0
dtype: int64

## 3-1. 라이브 부스트 효과 제거

In [130]:
# 라이브 부스트 효과 받는 row 추출
live_boost_data = data[(data['overall'].str.contains('\n'))]
live_boost_data 

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
50861,이재성,20,100\n(99),94,98,83,105,22KFA,"['2018 ~ 2021 / 1. FSV 마인츠 05', '2014 ~ 2018 /..."
50877,정우영,17,91\n(93),95,95,86,85,22KFA,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."
50881,정우영,5,67\n(66),77,76,52,83,LIVE,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."


In [131]:
# 라이브 부스트 효과 제거 및 data 
# 업데이트
for num in tqdm(range(len(live_boost_data))):
    gap, original_overall = find_gap(live_boost_data,num)
    
    remove_live_boost(live_boost_data,original_overall,gap,num)

data.update(live_boost_data)
data

  0%|          | 0/3 [00:00<?, ?it/s]

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,슈테펜 호프만,5,62,39,51,59,38,17,-
1,J. 피에라치,5,60,59,55,68,66,18,-
2,누어 허신,5,55,56,61,55,66,19,-
3,마오하오위,5,49,69,60,35,49,20,-
4,마이크 고메스,5,57,65,61,67,70,19,-
...,...,...,...,...,...,...,...,...,...
50878,정우영,14,86,94,98,73,83,19NG,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."
50879,정우영,14,83,73,67,86,87,2019KFA,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."
50880,정우영,9,76,88,88,64,75,21PLA,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."
50881,정우영,5,66,76,75,51,82,LIVE,"['2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '..."


In [132]:
# 중복 제거 및 csv 파일로 저장
data = drop_duplicate(data)
data.to_csv("./data/player_info/live_boost_off.csv", encoding='utf-8-sig', index = False)

## 3-2. 클럽 커리어 정리

In [138]:
# csv 파일 불러오기
data = pd.read_csv('./data/player_info/live_boost_off.csv')

In [140]:
# 클럽 커리어 정리
data, data_same_name = mk_club_career(data)
data

  0%|          | 0/50883 [00:00<?, ?it/s]

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,슈테펜 호프만,5,62,39,51,59,38,17,-
1,J. 피에라치,5,60,59,55,68,66,18,-
2,누어 허신,5,55,56,61,55,66,19,-
3,마오하오위,5,49,69,60,35,49,20,-
4,마이크 고메스,5,57,65,61,67,70,19,-
...,...,...,...,...,...,...,...,...,...
50878,정우영,14,86,94,98,73,83,19NG,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
50879,정우영,14,83,73,67,86,87,2019KFA,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
50880,정우영,9,76,88,88,64,75,21PLA,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
50881,정우영,5,66,76,75,51,82,LIVE,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."


In [ ]:
############################################################################################################

In [198]:
data_temp = data

In [204]:
temp_name_list = []

b = data_temp.name.value_counts() > 1

for i in range(len(data_temp)):
    if b.iloc[i] == True:
        temp_name_list.append(b.index[i])
    else:
        break
        
len(temp_name_list)

8159

In [205]:
# 추출된 선수들 중 data상의 존재 유무 표기
data_temp['OX'] = pd.Series()
    
for i in tqdm(range(len(data_temp.name))):
    # 동명이인 선수 중 크롤링 된 결과가 있는 데이터 >>> 클럽 경력이 잘못 입력되었을 가능성 높음 (수정 필요)
    if data_temp.name.iloc[i] in temp_name_list:
        data_temp['OX'].iloc[i] = 'O'
    
    # 동명이인 선수가 없는 선수 목록    
    elif data_temp.name.iloc[i] not in temp_name_list:
        data_temp['OX'].iloc[i] = 'X'
        
    else:
        data_temp['OX'].iloc[i] = 'error'
        


  0%|          | 0/50883 [00:00<?, ?it/s]

In [206]:
data_temp = data_temp[data_temp.OX == 'O']
data_temp

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career,OX
6,보이치에흐 슈체스니,20,105,68,72,79,63,LOL,"'2009 ~ 2017 / 유벤투스', '2015 ~ 2017 / 아스널', '임대...",O
7,보이치에흐 슈체스니,19,102,65,73,78,58,BTB,"'2009 ~ 2017 / 유벤투스', '2015 ~ 2017 / 아스널', '임대...",O
8,보이치에흐 슈체스니,18,100,63,71,75,56,21TOTS,"'2009 ~ 2017 / 유벤투스', '2015 ~ 2017 / 아스널', '임대...",O
9,보이치에흐 슈체스니,18,100,64,69,76,57,WC22,"'2009 ~ 2017 / 유벤투스', '2015 ~ 2017 / 아스널', '임대...",O
10,보이치에흐 슈체스니,17,98,50,68,68,48,20TOTS,"'2009 ~ 2017 / 유벤투스', '2015 ~ 2017 / 아스널', '임대...",O
...,...,...,...,...,...,...,...,...,...,...
50878,정우영,14,86,94,98,73,83,19NG,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2...",O
50879,정우영,14,83,73,67,86,87,2019KFA,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2...",O
50880,정우영,9,76,88,88,64,75,21PLA,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2...",O
50881,정우영,5,66,76,75,51,82,LIVE,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2...",O


In [224]:
temp_name_list[8157]

'다니엘 안옘베'

In [225]:
data_temp.name.iloc[0]

'보이치에흐 슈체스니'

In [ ]:
c = []

for i in tqdm(len(data_temp)):
    for j in range(len(data_temp)):
        if (data_temp.name.iloc[i] == data_temp.name.iloc[j]) & (data_temp.season.iloc[i] == data_temp.season.iloc[j]) & (data_temp.pay_side.iloc[i] != data_temp.pay_side.iloc[j]) & (data_temp.overall.iloc[i] != data_temp.overall.iloc[j]) & (data_temp.Sprint_speed.iloc[i] != data_temp.Sprint_speed.iloc[j]):
            c.append(data_temp.index(i))

c

In [210]:
data_temp.iloc[0]

name                                                   보이치에흐 슈체스니
pay_side                                                       20
overall                                                       105
Sprint_speed                                                   68
Acceleration                                                   72
Strength                                                       79
Stamina                                                        63
season                                                        LOL
club_career     '2009 ~ 2017 / 유벤투스', '2015 ~ 2017 / 아스널', '임대...
OX                                                              O
Name: 6, dtype: object

In [211]:
data_temp.index[0]

6

In [ ]:
# 이름까지 똑같은 동명이인 찾기 >>> 똑같은 이름에 시즌 여러개인 데이터 찾기
data.name[(data.name.value_counts() > 1) & (data.season.value_counts() > 1)]


## 동명이인 재 검색

# 상세 검색 클릭
# 이름 입력
# 최대/최소 오버롤 값에 오버롤 값 입력
# 최대/최소 급여 값에 급여값 추가
# 세부 능력 1에 '속력' 클릭
# 세부 능력 최대/최소 값에 속력 값 추가  * 반복 (가속력, 몸싸움)
# 검색버튼 클릭
# 세부정보 버튼 클릭
# 창변환
# 클럽경력 저장
# 창변환
# 초기화

In [ ]:
############################################################################################################

In [110]:
# 중복 제거 및 csv 파일로 저장
data = drop_duplicate(data)
data.to_csv("./data/player_info/remake_club_career.csv", encoding='utf-8-sig', index = False)

# 4. 데이터 수정 및 부족한 선수 데이터 추가

## 4-1. 재검색 필요한 선수명 찾기

In [115]:
# csv 파일 불러오기
data = pd.read_csv('./data/player_info/remake_club_career.csv')

### 4-1-1. 동명이인 1차 업데이트 부분 제거

> '동명이인' 포함된 컬럼 제거

In [118]:
def condition_1(data_same_name):
    # club_career에 '동명이인' 단어가 포함된 선수 찾기
    same_name_group = data_same_name[data_same_name.club_career.str.contains('동명이인')]

    # '동명이인'단어 지워주기
    same_name_group.club_career = same_name_group.club_career.str[4:]

    # 동명이인 1차 업데이트 데이터 제거
    condition_1 = same_name_group[same_name_group.club_career == ''].index
    same_name_group.drop(condition_1, inplace = True)

    return same_name_group

### 4-1-2. 동명이인 2차 업데이트 부분 제거

> '동명이인'을 빈칸으로 바꿨을 때, 남은 이름과 name이 같은 경우 제거

In [119]:
def condition_2(same_name_group):
    # 동명이인 2차 업데이트 데이터 제거
    condition_2 = same_name_group[same_name_group.club_career == same_name_group.name ].index
    same_name_group.drop(condition_2, inplace = True)
    
    return same_name_group

### 4-1-3. 동명이인 3차 업데이트 부분 수정

> 1 & 2차 업데이트 데이터를 제외한 데이터의 club_career의 리스트를 뽑아 중복값을 제거하여 잘못 입력된 선수 데이터들을 찾는다.

In [120]:
# 다시 검색이 필요한 선수 목록 찾는 함수
def mk_re_search_list(same_name_group_2,data):
    # 동명이인과 같이 검색된 선수명 리스트 추출
    same_name_list = list(set(same_name_group_2.club_career))

    # 추출된 선수들 중 data상의 존재 유무 표기
    data['OX'] = pd.Series()

    for i in tqdm(range(len(data.name))):
        # 동명이인 선수 중 크롤링 된 결과가 있는 데이터 >>> 클럽 경력이 잘못 입력되었을 가능성 높음 (수정 필요)
        if data.name.iloc[i] in same_name_list:
            data['OX'].iloc[i] = 'O'
        
        # 동명이인 선수가 없는 선수 목록    
        elif data.name.iloc[i] not in same_name_list:
            data['OX'].iloc[i] = 'X'
            
        else:
            data['OX'].iloc[i] = 'error'
            

    # 현재 존재하는 선수 목록 추출
    exist_same_name_data = list(set(data[data['OX'] == 'O'].name))


    # 재검색 목록
    re_search_list = []

    for name in same_name_list:
        if name not in exist_same_name_data:
            re_search_list.append(name)

    return re_search_list

## 4-2. 재검색

##### * 사용할 함수

In [70]:
# 크롬 윈도우 설정
def open_chrome():
    # 크롬 드라이버 위치 설정
    chrome_path = chromedriver_autoinstaller.install()


    # 크롬 옵션
    options = webdriver.ChromeOptions()
    # 크롬 윈도우 사이즈 조절
    options.add_argument("--window-size=800,1400")

    driver = webdriver.Chrome(chrome_path, options=options)

    # 실행할 시간 랜덤값 지정
    A = np.random.randint(5,7)
    
    return driver , A



# 클럽 활동 연도 & 클럽 팀 합치기
def year_and_team_merge(temp_club_career):
    global temp_club_career_2
    temp_club_career_2 = []
    year = []
    team = []
    

    for i in range(1,len(temp_club_career.split('\n'))):
        if i % 2 == 0:
            year.append(temp_club_career.split('\n')[i])
        else:
            team.append(temp_club_career.split('\n')[i])


    for i in range(len(year)):
        temp_club_career_2.append(year[i]+' / '+team[i])
        
    return temp_club_career_2



# 크롤링
def crawling(re_search_list,overall_num):
   # chrome 창 열기
   driver, A = open_chrome()

   ## 선수 정보 >> 각 선수의 시즌 & 클럽 경력 크롤링
   # 선수 정보 리스트
   name = []
   season = []
   club_career = []
   pay_side = []
   overall = []
   Sprint_speed = []
   Acceleration =[]
   Strength = []
   Stamina = []
   error_name = []

   # 홈페이지 열기
   url = 'https://fifaonline4.nexon.com/datacenter'
   driver.get(url)

   # # 팝업창 닫기
   # driver.find_element_by_xpath('//*[@id="wrapper"]/div[1]/a/span').click()
   # time.sleep(A)

   # 정지된 위치부터 다시 시작 
   for search_name in tqdm(re_search_list):
      try:
         # 선수명 입력 & 검색
         driver.find_element_by_class_name('ui-autocomplete-input').click()
         driver.find_element_by_class_name('ui-autocomplete-input').send_keys(search_name)
         driver.find_element_by_xpath('//*[@id="slider1"]/input[2]').clear() # 오버롤 숫자 지우기
         driver.find_element_by_xpath('//*[@id="slider1"]/input[2]').send_keys(f'{overall_num}') # 오버롤 설정
         driver.find_element_by_class_name('btn_search').click()
         time.sleep(2.5)


         # 세부 설명 들어가기
         driver.find_element_by_class_name('name').click()
         driver.find_element_by_class_name('btn_detail_link').click()
         time.sleep(0.5)
         
         
         # 창 변경
         driver.switch_to.window(driver.window_handles[-1])
         time.sleep(1)
         
         
         # 클럽 경력 추출
         temp_club_career = driver.find_element_by_xpath('//*[@id="middle"]/div/div/div[4]/div[1]/div[2]/div[2]').text
         temp_club_career_2 = year_and_team_merge(temp_club_career)
         
         
         # 세부 설명 나오기
         driver.close()
         time.sleep(0.5)

         
         # 창 변경
         driver.switch_to.window(driver.window_handles[0])
         time.sleep(0.5)



         # 검색된 선수 숫자 세기
         player_list = driver.find_elements_by_xpath('//*[@id="divPlayerList"]')
         player_list_num = player_list[0].text.count(f'{search_name}')



         # 시즌, 이름, 급여, 오버롤, 속력, 가속력, 몸싸움, 스태미너 추출
         for num in range(1,player_list_num+1):
            # 이름 저장
            temp_name = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[1]/div/div[4]/div[2]').text
            name.append(temp_name)
            
            # 클럽 경력 저장
            # 같이 검색된 동명이인 처리
            if search_name == temp_name:
               club_career.append(temp_club_career_2)
            else:
               club_career.append('동명이인'+search_name)

            # 시즌 저장
            temp_season = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[1]/div/div[4]/div[1]/img')
            temp_season = temp_season.get_attribute('src').split('/')[-1].replace('.png','')
            season.append(temp_season)
      
            # 급여 저장
            temp_pay_side = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[2]/span').text
            pay_side.append(temp_pay_side)
         
            # 오버롤 저장
            temp_overall = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[3]/span/span').text
            overall.append(temp_overall)
            
            # 속력 저장
            temp_Sprint_speed = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[4]/span/span').text
            Sprint_speed.append(temp_Sprint_speed)
            
            # 가속력 저장
            temp_Acceleration = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[5]/span/span').text
            Acceleration.append(temp_Acceleration)
            
            # 몸싸움 저장
            temp_Strength = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[6]/span/span').text
            Strength.append(temp_Strength)
            
            # 스태미너 저장
            temp_Stamina = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[7]/span/span').text
            Stamina.append(temp_Stamina)
         

         # 검색어 초기화 
         driver.find_element_by_class_name('btn_reset').click()
         time.sleep(1)
      
      except:
         if driver.find_element_by_xpath('//*[@id="divPlayerList"]/div/span').text == '검색 결과가 없습니다.':
            error_name.append(search_name)
            print(f'error(검색 결과가 없습니다.): {search_name}')
            
            # 검색어 초기화 
            driver.find_element_by_class_name('btn_reset').click()
            pass
         
         elif driver.find_element_by_xpath('//*[@id="ErrorMessage"]').text == '사용할 수 없는 페이지입니다.':
            error_name.append(search_name)
            print(f'error(사용할 수 없는 페이지입니다.): {search_name}')
            
            # 창 닫기
            driver.close()
            time.sleep(0.5)

            # 창 변경
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(0.5)
            
            # 검색어 초기화 
            driver.find_element_by_class_name('btn_reset').click()
            pass
            
         else:
            error_name.append(search_name)
            print(f'error: {search_name}')
            break

   return name,season,club_career,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,error_name



# dict로 변형 & 데이터 프레임화
def dict_to_df(name,season,club_career,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina):
    dict_temp = {  'name': [val for val in name],
                'pay_side': [val for val in pay_side],
                'overall': [val for val in overall],
                'Sprint_speed': [val for val in Sprint_speed],
                'Acceleration': [val for val in Acceleration],
                'Strength': [val for val in Strength],
                'Stamina': [val for val in Stamina],
                'season': [val for val in season],
                'club_career': [val for val in club_career]                            
    }
    columns = ['name','pay_side','overall','Sprint_speed','Acceleration','Strength','Stamina','season','club_career']

    player_info_plus = pd.DataFrame(data = dict_temp, columns=columns)
    
    return player_info_plus
 
 
 
def merge_csv_2(player_info_plus):
    # 이전 csv 파일 불러오기
    player_info_plus_before = pd.read_csv('./data/player_info/remake_club_career.csv')


    # 데이터 프레임 병합
    # 이전 csv 파일 불러오기
    New_player_info_plus = pd.concat([player_info_plus_before,player_info_plus], ignore_index=True)

    # csv 파일로 저장
    New_player_info_plus.to_csv("./data/player_info/player_info_plus_final.csv", encoding='utf-8-sig', index = False)
    
    return New_player_info_plus  

##### 실행

In [121]:
# 재검색 할 선수 목록
same_name_group = condition_1(data_same_name)
same_name_group_2 = condition_2(same_name_group)
re_search_list = mk_re_search_list(same_name_group_2,data)
len(re_search_list)

  0%|          | 0/50860 [00:00<?, ?it/s]

19

In [126]:
re_search_list = ['이재성','정우영']

In [122]:
# 직접 추가하고싶은 선수 추가
add_list = ['호나우두']
re_search_list = re_search_list + add_list
len(re_search_list)

20

In [127]:
# 재검색
overall_num = 180
name,season,club_career,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,error_name = crawling(re_search_list,overall_num)
player_info_plus = dict_to_df(name,season,club_career,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina)
data = merge_csv(player_info_plus) # 예비용 저장
data_final = merge_csv_2(player_info_plus)
data_final

  0%|          | 0/2 [00:00<?, ?it/s]

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
0,슈테펜 호프만,5,62,39,51,59,38,17,-
1,J. 피에라치,5,60,59,55,68,66,18,-
2,누어 허신,5,55,56,61,55,66,19,-
3,마오하오위,5,49,69,60,35,49,20,-
4,마이크 고메스,5,57,65,61,67,70,19,-
...,...,...,...,...,...,...,...,...,...
50878,정우영,14,86,94,98,73,83,19NG,"[2018 ~ 2018 / 알사드, 2016 ~ 2017 / 비셀 고베, 2014 ..."
50879,정우영,14,83,73,67,86,87,2019KFA,"[2018 ~ 2018 / 알사드, 2016 ~ 2017 / 비셀 고베, 2014 ..."
50880,정우영,9,76,88,88,64,75,21PLA,"[2018 ~ 2018 / 알사드, 2016 ~ 2017 / 비셀 고베, 2014 ..."
50881,정우영,5,67\n(66),77,76,52,83,LIVE,"[2018 ~ 2018 / 알사드, 2016 ~ 2017 / 비셀 고베, 2014 ..."


In [128]:
data_final[data_final.name == '정우영']

,name,pay_side,overall,Sprint_speed,Acceleration,Strength,Stamina,season,club_career
46778,정우영,18,98,88,91,100,100,2012KH,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
46779,정우영,17,96,89,90,97,98,22KFA,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
46780,정우영,17,93,97,97,88,87,22KFA,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
46781,정우영,14,86,94,98,73,83,19NG,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
46782,정우영,14,83,73,67,86,87,2019KFA,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
46783,정우영,9,76,88,88,64,75,21PLA,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
46784,정우영,5,66,76,75,51,82,LIVE,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
46785,정우영,5,56,72,74,54,56,17,"'2018 ~ 2018 / 알사드', '2016 ~ 2017 / 비셀 고베', '2..."
50858,정우영,5,66,76,75,51,82,LIVE,"'2020 ~ 2020 / SC 프라이부르크', '임대 / FC 바이에른 뮌헨 II..."
50859,정우영,5,56,72,74,54,56,17,"'2020 ~ 2020 / SC 프라이부르크', '임대 / FC 바이에른 뮌헨 II..."


> 3.데이터 후처리로 이동 & 반복